In [6]:
import os

In [7]:
%pwd

'e:\\GCET\\Machine Learning\\Deep Learning Projects\\text_recognition\\research'

In [8]:
os.chdir('../')

In [9]:
%pwd

'e:\\GCET\\Machine Learning\\Deep Learning Projects\\text_recognition'